In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

# Distribution de la température dans un appartement d'un immeuble aux plusieurs étages


# Équation de transfert de chaleur:
# k*(d^2 T(x,y)/dx^2 + d^2 T(x,y)/dy^2)+S=0


# Conditions aux limites:

# (1) Condition convective (de Robin) à x=0 et à x=Lx (faces externes du mur):
# -k*dT(x=0,y)/dx=-h*(T-Ta)
# -k*dT(x=L,y)/dx=h*(T-Ta)
Ta=-20; #oC

# (2) Condition de Dirichlet sur le plafond et sur le plancher
# T(x, y=0 ou y=Ly)=Tp
Tp=20; #oC

# Dimensions d'appartement
Lx=4; #[m]
Ly=2.4;  #[m]

# Parametres d'un mur d'isolation thermique
Lm=0.4; #m ; Épaisseur du mur en brique
km=0.85;#W/(m*K); La conductivité thermique de la brique
h=10; #W/(m^2*K); Coefficient de transfert thermique sur les surfaces extérieures du mur

# Paramètres de l'air qui remplit l'appartement
ka=0.024;

fact_ar = np.array([2.0, 1.0, 0.5, 0.25], dtype=np.double); # Matrice pleine
d_ar=np.zeros(fact_ar.size,dtype=np.double);
tini_ar=np.zeros(fact_ar.size,dtype=np.double);
tinv_ar=np.zeros(fact_ar.size,dtype=np.double);
mem_ar=np.zeros(fact_ar.size,dtype=np.double);
Tm_ar=np.zeros(fact_ar.size,dtype=np.double);
Err_ar=np.zeros(fact_ar.size-1,dtype=np.double);
d_Err_ar=np.zeros(fact_ar.size-1,dtype=np.double);

ci=-1;
for fact in fact_ar:
    ci=ci+1;
    d=0.1*fact;    #Pas de discrétisation en [m]
    print('Pas de discrétisation dx=dy=',d,'m')
    d_ar[ci]=d;
    Nx=int(np.rint(Lx/d+1)); # Nombre de nœuds le long de X
    Ny=int(np.rint(Ly/d+1)); # Nombre de nœuds le long de Y
    
    
    tic=time.time_ns();
    
    # Initialisation de la source de chaleur, de la conductivité thermique et de la matrice
    S=np.zeros((Ny,Nx),dtype=np.double);
    k=np.zeros((Ny,Nx),dtype=np.double);
    for i in np.arange(1,Ny+1,1): #i=1,..,Ny - numérotation des nœuds sur un maillage physique
        y=(i-1)*d;
        for j in np.arange(1,Nx+1,1): #j=1,..,Nx - numérotation des nœuds sur un maillage physique
            x=(j-1)*d;
            
            # Source volumique de chaleur q[W/m^3] d'épaisseur dL.
            # La source est intégrée dans les parties intérieures du mur à x=Lm et à x=Lx-Lm et
            # il occupe les tiers du mur dans la direction verticale
            dL=0.1;
            q=2.0e3;# W/m^3;
            if ((x<=Lm) and (y<=Ly/3+Lm) and (y>Lm)):
                # À l'intérieur de l'élément chauffant
                S[i-1,j-1]=q*np.exp(-((x-Lm)/dL)**2);
            elif ((x>=(Lx-Lm)) and (y<=Ly/3+Lm) and (y>Lm)):
                # À l'intérieur de l'élément chauffant
                S[i-1,j-1]=q*np.exp(-((Lx-Lm-x)/dL)**2);
            else:
                # À l'extérieur de l'élément chauffant
                S[i-1,j-1]=0.0;
            
            # L'espace de vie de l'appartement est délimité par
            # les parois d'épaisseur Lm à tous les quatre côtés
            if ((x<=Lm) or (x>=(Lx-Lm)) or (y<=Lm) or (y>=(Ly-Lm))):
                # À l'intérieur du mur
                k[i-1,j-1]=km;
            else:
                # À l'intérieurde de l'appartement
                k[i-1,j-1]=ka;
               
    M=np.zeros((Nx*Ny,Nx*Ny),dtype=np.double);
    b=np.zeros((Nx*Ny,1),dtype=np.double);
    T=np.zeros((Nx*Ny,1),dtype=np.double);
    Tr=np.zeros((Ny,Nx),dtype=np.double);
    
    for i in np.arange(1,Ny+1,1):
        for j in np.arange(1,Nx+1,1):
            # remplir la ligne pl de la matrice M
            pl=(i-1)*Nx+j;
            x = (j-1)*d;
            y = (i-1)*d;
            
            if (i==1):
                # noeud sur le plafond y=0
                pc=pl;M[pl-1,pc-1]=1; # contribution de noeud (1,j)
                b[pl-1]=Tp;
            elif (i==Ny):
                # noeud sur le plancher y=Ly
                pc=pl;M[pl-1,pc-1]=1; # contribution de noeud (Ny,j)
                b[pl-1]=Tp;
            elif (j==1):
                # noeud à la surface externe du mur x=0
                pc=pl;M[pl-1,pc-1]=3+2*d*h/k[i-1,j-1]; # contribution de noeud (i,1)
                pc=(i-1)*Nx+j+1;M[pl-1,pc-1]=-4; # contribution de noeud (i,2)
                pc=(i-1)*Nx+j+2;M[pl-1,pc-1]=1; # contribution de noeud (i,3)
                b[pl-1]=2*d*h*Ta/k[i-1,j-1];
            elif (j==Nx):
                # noeud à la surface externe du mur x=Nx
                pc=pl;M[pl-1,pc-1]=3+2*d*h/k[i-1,j-1]; # contribution de noeud (i,Nx)
                pc=(i-1)*Nx+j-1;M[pl-1,pc-1]=-4; # contribution de noeud (i,Nx-1)
                pc=(i-1)*Nx+j-2;M[pl-1,pc-1]=1; # contribution de noeud (i,Nx-2)
                b[pl-1]=2*d*h*Ta/k[i-1,j-1];
            elif ((x==Lm or x==Lx-Lm) and (y<=Ly-Lm and y>=Lm)):
                if x==Lm:
                    sens = 1
                else : 
                    sens = -1
                pc=pl;M[pl-1,pc-1]=3*(km+ka)
                pc=pl+sens;M[pl-1,pc-1]=-4*ka
                pc=pl-sens;M[pl-1,pc-1]=-4*km
                pc=pl+2*sens;M[pl-1,pc-1]=ka
                pc=pl-2*sens;M[pl-1,pc-1]=km
                b[pl-1]=0


            elif ((y==Lm or y==Ly-Lm) and (x<=Lx-Lm and x>=Lm)):
                if y==Lm:
                    sens = 1
                else : 
                    sens = -1
                pc=pl;M[pl-1,pc-1]=3*(km+ka)
                pc=(i-1+sens)*Nx+j;M[pl-1,pc-1]=-4*ka
                pc=(i-1-sens)*Nx+j;M[pl-1,pc-1]=-4*km
                pc=(i-1+2*sens)*Nx+j;M[pl-1,pc-1]=ka
                pc=(i-1-2*sens)*Nx+j;M[pl-1,pc-1]=km
                b[pl-1]=0

                
            else:
                # noeud qui est strictement à l'intérieur de la cellule de simulation
                pc=pl;M[pl-1,pc-1]=-4; # contribution de noeud (i,j)
                pc=(i-1)*Nx+j-1;M[pl-1,pc-1]=1; # contribution de noeud (i,j-1)
                pc=(i-1)*Nx+j+1;M[pl-1,pc-1]=1; # contribution de noeud (i,j+1)
                pc=(i-2)*Nx+j;M[pl-1,pc-1]=1; # contribution de noeud (i-1,j)
                pc=(i)*Nx+j;M[pl-1,pc-1]=1; # contribution de noeud (i+1,j)
                b[pl-1]=-d**2*S[i-1,j-1]/k[i-1,j-1];




    toc=time.time_ns();
    tini_ar[ci]=(toc-tic)/1.0e9; #temps en [s]  


    tic=time.time_ns();
    T=np.linalg.solve(M,b);
    toc=time.time_ns();
    tinv_ar[ci]=(toc-tic)/1.0e9; #temps en [s]  
    
    mem_ar[ci]=2*8*(Nx*Ny)**2;
    print('Mémoire nécessaire pour stocker les matrices L et U',round(mem_ar[ci]),'[octet]');
    Tr=np.reshape(T,(Ny,Nx),order='C');
    
    Tm_ar[ci]=Tr[int(np.rint(Ly/d/2+1))-1,int(np.rint(Lx/d/2+1))-1]; # température au milieu du domaine de calcul

#plt.figure(1)
#plt.pcolor(np.arange(0,Nx,1)*d,np.arange(0,Ny,1)*d,S);
#plt.colorbar(mappable=None, cax=None, ax=None);
#plt.title('S(x,y) [W/$m^3$]')
#plt.xlabel('x [m]')    
#plt.ylabel('y [m]')


#plt.figure(2)
#plt.pcolor(np.arange(0,Nx,1)*d,np.arange(0,Ny,1)*d,k);
#plt.colorbar(mappable=None, cax=None, ax=None);
#plt.title('k(x,y) [W/($m^2\cdot$K)]')
#plt.xlabel('x [m]')    
#plt.ylabel('y [m]')

plt.figure(3)
#plt.title(f'T(x,y) [$^o$C] — pas de discrétisation d = {d:.3f} m')
plt.pcolor(np.arange(0, Nx, 1)*d, np.arange(0, Ny, 1)*d, Tr)
plt.colorbar(mappable=None, cax=None, ax=None)
plt.xlabel('x [m]', fontsize=16)
plt.ylabel('y [m]', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.savefig("Img123/T.png")
plt.close()


plt.figure(4)
plt.loglog(d_ar[::-1],mem_ar[::-1]/1024.0**3,'-o')
#plt.title('Exigences de mémoire')
plt.xlabel('Pas $d_x=d_y$ [m]', fontsize=16)
plt.ylabel('Mémoire [Gb]', fontsize=16)
plt.savefig("Img123/Ram")
plt.close()


plt.figure(5)
Err_ar=abs(Tm_ar[:-1:]-Tm_ar[1::])
d_Err_ar=d_ar[1::]; # Definiton d'erreur Err(delta)=|Tm(2*delta)-Tm(delta)|
plt.loglog(d_Err_ar[::-1],Err_ar[::-1],'-o')
#plt.title('Erreur de calcul')
plt.xlabel('Pas $d_x=d_y$ [m]', fontsize=16)
plt.ylabel('Err [$^o$C]', fontsize=16)
plt.savefig("Img123/Erreur.png")
plt.close()

plt.figure(6)
plt.loglog(d_ar[::-1],tini_ar[::-1],'-bo',d_ar[::-1],tinv_ar[::-1],'-r*')
plt.title('Temps de calcul(initialisation et inversion)')
plt.xlabel('Pas $d_x=d_y$ [m]')
plt.ylabel('t [s]')
plt.legend(['$t_{initialisation}$','$t_{inversion}$'])
plt.savefig("Img123/Temps.png")
plt.close()

p = np.polyfit(np.log(d_Err_ar[::-1]), Err_ar[::-1], 1)
P_err = p[0]
A_err = np.exp(p[1])
print(f"Le coefficient de l'erreur est {A_err}")
print(f"L'exposant de l'erreur est {P_err}")

p = np.polyfit(np.log(d_ar[::-1]), np.log(mem_ar[::-1]), 1)
exp_mem = p[0]
Coeff_mem= np.exp(p[1])
print(f"Le coefficient de la mémoire est {Coeff_mem}")
print(f"L'exposant de la mémoire est {exp_mem}")

In [ ]:
import time
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

# Distribution de la température dans un appartement d'un immeuble aux plusieurs étages


# Équation de transfert de chaleur:
# k*(d^2 T(x,y)/dx^2 + d^2 T(x,y)/dy^2)+S=0


# Conditions aux limites:

# (1) Condition convective (de Robin) à x=0 et à x=Lx (faces externes du mur):
# -k*dT(x=0,y)/dx=-h*(T-Ta)
# -k*dT(x=L,y)/dx=h*(T-Ta)
Ta=-20; #oC

# (2) Condition de Dirichlet sur le plafond et sur le plancher
# T(x, y=0 ou y=Ly)=Tp
Tp=20; #oC

# Dimensions d'appartement
Lx=4; #[m]
Ly=2.4;  #[m]

# Parametres d'un mur d'isolation thermique
Lm=0.4; #m ; Épaisseur du mur en brique
km=0.85;#W/(m*K); La conductivité thermique de la brique
h=10; #W/(m^2*K); Coefficient de transfert thermique sur les surfaces extérieures du mur

# Paramètres de l'air qui remplit l'appartement
ka=0.024;

fact_ar = np.array([1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625], dtype=np.double); # Matrice pleine
d_ar=np.zeros(fact_ar.size,dtype=np.double);
tini_ar=np.zeros(fact_ar.size,dtype=np.double);
tinv_ar=np.zeros(fact_ar.size,dtype=np.double);
mem_ar=np.zeros(fact_ar.size,dtype=np.double);
Tm_ar=np.zeros(fact_ar.size,dtype=np.double);
Err_ar=np.zeros(fact_ar.size-1,dtype=np.double);
d_Err_ar=np.zeros(fact_ar.size-1,dtype=np.double);

ci=-1;
for fact in fact_ar:
    ci=ci+1;
    d=0.1*fact;    #Pas de discrétisation en [m]
    print('Pas de discritization dx=dy=',d,'m')
    d_ar[ci]=d;
    Nx=int(np.rint(Lx/d+1)); # Nombre de nœuds le long de X
    Ny=int(np.rint(Ly/d+1)); # Nombre de nœuds le long de Y
    
    tic=time.time_ns();
    
    # Initialisation de la source de chaleur, de la conductivité thermique et de la matrice
    S=np.zeros((Ny,Nx),dtype=np.double);
    k=np.zeros((Ny,Nx),dtype=np.double);
    for i in np.arange(1,Ny+1,1): #i=1,..,Ny - numérotation des nœuds sur un maillage physique
        y=(i-1)*d;
        for j in np.arange(1,Nx+1,1): #j=1,..,Nx - numérotation des nœuds sur un maillage physique
            x=(j-1)*d;
            
            # Sourse volumique de chaleur q[W/m^3] d'épaisseur dL.
            # La source est intégrée dans les parties intérieures du mur à x=Lm et à x=Lx-Lm et
            # il occupe les tiers du mur dans la direction verticale
            dL=0.1;
            q=2.0e3;# W/m^3;
            if ((x<=Lm) and (y<=Ly/3+Lm) and (y>Lm)):
                # À l'intérieur de l'élément chauffant
                S[i-1,j-1]=q*np.exp(-((x-Lm)/dL)**2);
            elif ((x>=(Lx-Lm)) and (y<=Ly/3+Lm) and (y>Lm)):
                # À l'intérieur de l'élément chauffant
                S[i-1,j-1]=q*np.exp(-((Lx-Lm-x)/dL)**2);
            else:
                # À l'extérieur de l'élément chauffant
                S[i-1,j-1]=0.0;
            
            # L'espace de vie de l'appartement est délimité par
            # les parois d'épaisseur Lm à tous les quatre côtés
            if ((x<=Lm) or (x>=(Lx-Lm)) or (y<=Lm) or (y>=(Ly-Lm))):
                # À l'intérieur du mur
                k[i-1,j-1]=km;
            else:
                # À l'intérieurde de l'appartement
                k[i-1,j-1]=ka;
               
    M=sp.sparse.lil_matrix((Nx*Ny,Nx*Ny), dtype=np.double); # Initialisation d'une matrice creuse
   
    b=np.zeros((Nx*Ny,1),dtype=np.double);
    T=np.zeros((Nx*Ny,1),dtype=np.double);
    Tr=np.zeros((Ny,Nx),dtype=np.double);

    for i in np.arange(1,Ny+1,1):
        for j in np.arange(1,Nx+1,1):
            # remplir la ligne pl de la matrice M
            pl=(i-1)*Nx+j;
            x = (j-1)*d;
            y = (i-1)*d;
            
            if (i==1):
                # noeud sur le plafond y=0
                pc=pl;M[pl-1,pc-1]=1; # contribution de noeud (1,j)
                b[pl-1]=Tp;
            elif (i==Ny):
                # noeud sur le plancher y=Ly
                pc=pl;M[pl-1,pc-1]=1; # contribution de noeud (Ny,j)
                b[pl-1]=Tp;
            elif (j==1):
                # noeud à la surface externe du mur x=0
                pc=pl;M[pl-1,pc-1]=3+2*d*h/k[i-1,j-1]; # contribution de noeud (i,1)
                pc=(i-1)*Nx+j+1;M[pl-1,pc-1]=-4; # contribution de noeud (i,2)
                pc=(i-1)*Nx+j+2;M[pl-1,pc-1]=1; # contribution de noeud (i,3)
                b[pl-1]=2*d*h*Ta/k[i-1,j-1];
            elif (j==Nx):
                # noeud à la surface externe du mur x=Nx
                pc=pl;M[pl-1,pc-1]=3+2*d*h/k[i-1,j-1]; # contribution de noeud (i,Nx)
                pc=(i-1)*Nx+j-1;M[pl-1,pc-1]=-4; # contribution de noeud (i,Nx-1)
                pc=(i-1)*Nx+j-2;M[pl-1,pc-1]=1; # contribution de noeud (i,Nx-2)
                b[pl-1]=2*d*h*Ta/k[i-1,j-1];
            elif ((x==Lm or x==Lx-Lm) and (y<=Ly-Lm and y>=Lm)):
                if x==Lm:
                    sens = 1
                else : 
                    sens = -1
                pc=pl;M[pl-1,pc-1]=3*(km+ka)
                pc=pl+sens;M[pl-1,pc-1]=-4*ka
                pc=pl-sens;M[pl-1,pc-1]=-4*km
                pc=pl+2*sens;M[pl-1,pc-1]=ka
                pc=pl-2*sens;M[pl-1,pc-1]=km
                b[pl-1]=0


            elif ((y==Lm or y==Ly-Lm) and (x<=Lx-Lm and x>=Lm)):
                if y==Lm:
                    sens = 1
                else : 
                    sens = -1
                pc=pl;M[pl-1,pc-1]=3*(km+ka)
                pc=(i-1+sens)*Nx+j;M[pl-1,pc-1]=-4*ka
                pc=(i-1-sens)*Nx+j;M[pl-1,pc-1]=-4*km
                pc=(i-1+2*sens)*Nx+j;M[pl-1,pc-1]=ka
                pc=(i-1-2*sens)*Nx+j;M[pl-1,pc-1]=km
                b[pl-1]=0

                
            else:
                # noeud qui est strictement à l'intérieur de la cellule de simulation
                pc=pl;M[pl-1,pc-1]=-4; # contribution de noeud (i,j)
                pc=(i-1)*Nx+j-1;M[pl-1,pc-1]=1; # contribution de noeud (i,j-1)
                pc=(i-1)*Nx+j+1;M[pl-1,pc-1]=1; # contribution de noeud (i,j+1)
                pc=(i-2)*Nx+j;M[pl-1,pc-1]=1; # contribution de noeud (i-1,j)
                pc=(i)*Nx+j;M[pl-1,pc-1]=1; # contribution de noeud (i+1,j)
                b[pl-1]=-d**2*S[i-1,j-1]/k[i-1,j-1];


    toc=time.time_ns();
    tini_ar[ci]=(toc-tic)/1.0e9; #temps en [s]  
    
    tic=time.time_ns();
    lu=sp.sparse.linalg.splu(sp.sparse.lil_matrix.tocsc(M));  
    T=lu.solve(b);
    #T=sp.sparse.linalg.spsolve(sp.sparse.lil_matrix.tocsc(M), b); #Matrice creuse
    toc=time.time_ns();
    tinv_ar[ci]=(toc-tic)/1.0e9; #temps en [s]  
    
    mem_ar[ci]=mem_ar[ci]=3*8*(lu.L.nnz + lu.U.nnz);
    print('le nombre d''éléments non nuls dans les matrices L+U:',round(mem_ar[ci]/8));
    
    Tr=np.reshape(T,(Ny,Nx),order='C');
    
    Tm_ar[ci]=Tr[int(np.rint(Ly/d/2+1))-1,int(np.rint(Lx/d/2+1))-1]; # température au milieu du domaine de calcul

#plt.figure(1)
#plt.pcolor(np.arange(0,Nx,1)*d,np.arange(0,Ny,1)*d,S);
#plt.colorbar(mappable=None, cax=None, ax=None);
#plt.title('S(x,y) [W/$m^3$]')
#plt.xlabel('x [m]')    
#plt.ylabel('y [m]')
#plt.savefig("Img456/S.pdf")
#plt.close()

#plt.figure(2)
#plt.pcolor(np.arange(0,Nx,1)*d,np.arange(0,Ny,1)*d,k);
#plt.colorbar(mappable=None, cax=None, ax=None);
#plt.title('k(x,y) [W/($m^2\\cdot$K)]')
#plt.xlabel('x [m]')    
#plt.ylabel('y [m]')
#plt.savefig("Img456/k.pdf")
#plt.close()

plt.figure(3)
plt.pcolor(np.arange(0,Nx,1)*d,np.arange(0,Ny,1)*d,Tr);
plt.colorbar(mappable=None, cax=None, ax=None);
plt.title('T(x,y) [$^o$C]')
plt.xlabel('x [m]', fontsize = 16)    
plt.ylabel('y [m]', fontsize = 16)
plt.savefig("Img456/T.png")
plt.close()

plt.figure(4)
plt.loglog(d_ar[::-1],mem_ar[::-1]/1024.0**3,'-o')
# plt.title('Exigences de mémoire')
plt.xlabel('Pas $d_x=d_y$ [m]', fontsize = 16)
plt.ylabel('Mémoire [Gb]', fontsize = 16)
plt.savefig("Img456/Ram.png")
plt.close()

plt.figure(5)
Err_ar=abs(Tm_ar[:-1:]-Tm_ar[1::]);
d_Err_ar=d_ar[1::]; # Definiton d'erreur Err(delta)=|Tm(2*delta)-Tm(delta)|
plt.loglog(d_Err_ar[::-1],Err_ar[::-1],'-o')
plt.title('Erreur de calcul')
plt.xlabel('Pas $d_x=d_y$ [m]')
plt.ylabel('Err [$^o$C]')
plt.savefig("Img456/Erreur.png")
plt.close()

plt.figure(6)
plt.loglog(d_ar[::-1],tini_ar[::-1],'-bo',d_ar[::-1],tinv_ar[::-1],'-r*')
plt.title('Temps de calcul(initialisation et inversion)')
plt.xlabel('Pas $d_x=d_y$ [m]')
plt.ylabel('t [s]')
plt.legend(['$t_{initialisation}$','$t_{inversion}$'])
plt.savefig("Img456/Temps.png")
plt.close()

print(f"Les différentes valeurs de Tm selon le pas sont: {Tm_ar[1::]}")
print(f"Les différentes valeurs de  l'erreur sur Tm selon le pas sont: {Err_ar[::-1]}")

p = np.polyfit(np.log(d_ar[::-1]), np.log(mem_ar[::-1]), 1)
exp_mem = p[0]
Coeff_mem= np.exp(p[1])
print(f"Amem est {Coeff_mem}")
print(f"Pmem est {exp_mem}")

Pas de discritization dx=dy= 0.1 m
le nombre déléments non nuls dans les matrices L+U: 79173
Pas de discritization dx=dy= 0.05 m
le nombre déléments non nuls dans les matrices L+U: 523605
Pas de discritization dx=dy= 0.025 m
le nombre déléments non nuls dans les matrices L+U: 3454161
Pas de discritization dx=dy= 0.0125 m
le nombre déléments non nuls dans les matrices L+U: 19525335
Pas de discritization dx=dy= 0.00625 m
le nombre déléments non nuls dans les matrices L+U: 106825833
Pas de discritization dx=dy= 0.003125 m
